In [ ]:
import numpy as np  # biblioteca utilizada para tratar com número/vetores/matrizes
import matplotlib.pyplot as plt  # utilizada para plotar gráficos ao "estilo" matlab
import pandas as pd #biblioteca utilizada para realizar operações sobre dataframes

In [ ]:
import io  #biblioteca utilizada para tratar os comandos de entrada e saida
data = pd.read_csv('AAPL.csv') # utilizado para importar o arquivo CSV que contém o banco de dados

In [ ]:
data.head() #comando utilizado para mostrar as 5 primeiras colunas do dataframe

In [ ]:
plt.plot(data["Date"],data["Open"])
plt.ylabel("Preço")
plt.xlabel("Data")

In [ ]:
#comando utilizado para gerar o boxplot
#boxplot é empregado para ver se existem outlier
data.boxplot(column='Open')

In [ ]:
#comando utilizado para verificar se existem dados nulos, numéricos ou não
data.info()

In [ ]:
#preparando os dados para a entrada
entradas=data.iloc[:,1:2].values # retira do dataframe apenas a coluna relativa ao preço de abertura das ações
entradas[:5] #utilizada para realiar um print apenas das 5 primeiras linhas (pode ser visto que esse é um array, não um dataframe)

In [ ]:
#normalizando os dados (os algoritmos de ML, em geral, não trabalham bem com dados em escalas diferentes)
from sklearn.preprocessing import MinMaxScaler  #biblioteca utilizada para realizar o preprocessamento dos dados
scaler = MinMaxScaler(feature_range = (0, 1))    #cria o objeto que será utilizado para realizar a normalização dos dados
                                                 # feaure_range = define o intervalo de escala dos dados

dados_normalizados = scaler.fit_transform(entradas)  # aplica o método de transformação dos dados

In [ ]:
dados_normalizados[:5]

In [ ]:
#Como as redes recorrentes utilizam dados no tempo T e valores passados (T-n), a entrada da rede deve conter os 
#valores presentes e os (T-n). Assim, é necessário realizar uma modificação nos dados
features_set = []  
labels = []  
for i in range(60, 1259):  
    features_set.append(dados_normalizados[i-60:i, 0])
    labels.append(dados_normalizados[i, 0])

In [ ]:
#transformando os dados em um array para serem utilizados como entrada
features_set, labels = np.array(features_set), np.array(labels)  

In [ ]:
#conferindo a dimensão dos dados
print(features_set.shape) # método utilizado para retornar a dimensão dos dados
print(labels.shape)

In [ ]:
#transformando os dados para o formato aceito pelas redes recorrentes do Keras
# 1 - formato em 3D
# (a,b,c) -> a = número de linhas do dataset
#         -> b = número de steps (entradas) da rede
#         -> c = número de saídas (indicators)

# método da biblioteca numpy que é utilizado para converter os dados de entrada (1199,60) em (1199,60,1)
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))  # 
print(features_set.shape)

In [ ]:
from keras.models import Sequential  #classe utilizada para criar o modelo sequencial utilizando o keras
from keras.layers import Dense  # Classe utilizada para criar as camadas que são completamente conectadas
from keras.layers import LSTM  # Classe para a rede recorrente utilizando Long Shor Term Memory
from keras.layers import Dropout # Classe utilizada para a camada de dropout (utilizada para evitar o overfiting)

In [ ]:
model = Sequential()  # objeto para a criação do modelo keras sequencial
model.add(LSTM(units=50, return_sequences=True, input_shape=(features_set.shape[1], 1)))  #cria a camada de entrada
# como pode ser visto, ela é adicionada como uma pilha, cada nova camada é adicionada com o método add
# na camada de entrada, é necessário definir o tamanho do vetor de entrada (input_shape)
# units=50 indica que na camada de entrada devem existir 50 neurônios
# return_sequences= True indica que devem ser adicionadas novas camadas


In [ ]:
#adição da camada de dropout
model.add(Dropout(0.2))  # o valor de 0.2 indica que 20% dos neurônios dessa camada serão perdidos para cada interação

In [ ]:
#Adicionando mais camadas ao modelo 
model.add(LSTM(units=50, return_sequences=True))  
model.add(Dropout(0.2))


model.add(LSTM(units=50, return_sequences=True))  
model.add(Dropout(0.2))

model.add(LSTM(units=50))  # como essa é a última camada LSTM utilizada, a variável return_sequences=False 
model.add(Dropout(0.2))  

In [ ]:
#adiciona a camada de saída com apenas 1 neurônio, pois vamos realizar a previsão de apenas uma variável (Previsão do valor de abertura)
model.add(Dense(units = 1))

In [ ]:
#comando utilizado para ver a configuração do nosso modelo
model.summary()

In [ ]:
#definição do tipo de função perda a ser utilizada e do tipo do otimizador 
# o otimizador é utilizado para minimizar a função perda
# a função perda indica como deve ser calculado o erro do modelo (valor real - valor previsto)
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
#treinamento do modelo
model.fit(features_set, labels, epochs = 100, batch_size = 32)  
# valores de entrada
# saída
# número épocas para o treinamento (vezes em que vamos realizar interações durante o treinamento)
#batch_size = quantidade de dados utilizados por vez para realizar o treinamento

In [ ]:
uploaded2=files.upload() #importa os arquivos para o teste dos dados 

In [ ]:
data_test = pd.read_csv(io.BytesIO(uploaded2['AAPL_previsao.csv'])) # utilizado para importar o arquivo CSV que contém o banco de dados para teste

In [ ]:
data_test.head() #verifica as 5 primeiras linhas do dataframe utilizado para teste

In [ ]:
plt.plot(data_test.Date,data_test.Open) #plot para os dados a serem utilizados como teste

In [ ]:
df_data_apple=data_test.iloc[:, 1:2].values

In [ ]:
df_data_apple = pd.concat((data['Open'], data_test['Open']), axis=0)  # concatena os dados utilizados para teste e os utilizados para treinamento, tudo em um mesmo dataframe de 1 coluna


In [ ]:
df_data_apple.head()

In [ ]:
test_inputs = df_data_apple[len(df_data_apple) - len(data_test) - 60:].values  


In [ ]:
#normalização dos dados para teste, como fizemos com os dados de treinamento
test_inputs = test_inputs.reshape(-1,1)  
test_inputs = scaler.transform(test_inputs)  

In [ ]:
#preparação dos 60 dados a setem utilizado
test_features = []  
for i in range(60, 80):  
    test_features.append(test_inputs[i-60:i, 0])

In [ ]:
#preparando os dados como entrada para o modelo de previsão 
test_features = np.array(test_features)  
test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1)) 

In [ ]:
#previsão utilizando o modelo gerado 
previsao = model.predict(test_features)  

In [ ]:
#inverte a transformação (normalização) dos dados de previsão
previsao = scaler.inverse_transform(previsao)

In [ ]:
#plot do resultado da previsão e do real
plt.figure(figsize=(10,6))  
plt.plot(data_test.Open, color='blue', label='Preço Real das Ações da Apple')  
plt.plot(previsao , color='red', label='Previsão do Preço das Ações da Apple')  
plt.title('Previsão do Preço de Abertura das Ações da Apple')  
plt.xlabel('Data')  
plt.ylabel('Preço')  
plt.legend()  
plt.show() 